In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import pyodbc
from datetime import datetime 
from tensorflow.keras.models import load_model
from flask import Flask

In [2]:
cnn = load_model('model.keras')

In [3]:
connection = pyodbc.connect("DSN=SQL")

In [4]:
cur = connection.cursor()

In [5]:
cur.execute('create table Attendence_Record ( Date DATE, Name Varchar(30) , Entry_Time Time)')

In [42]:
cap = cv2.VideoCapture(0)
results = []

while (True):
    ret,frame = cap.read()
    if ret == False:
        print("Can't capture an image.")
        break
    
    cv2.imshow('Camera',frame)
    key = cv2.waitKey(1)

    if key == ord('s'):

        cap_img= 'captured Image.jpg'.format(len(results) + 1)
        cv2.imwrite(cap_img,frame)
        resized_frame = cv2.resize(frame, (300, 300))
        img = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
        g_image = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        in_img = g_image.reshape(1,300,300,1)
        predection = cnn.predict(in_img)
        labels = ['donald','mickey','minion','olfa','pooh','pumba']
        Name = labels[np.argmax(predection)]


        currunt_time = datetime.now()
        Date = currunt_time.strftime('%Y-%m-%D')
        time = currunt_time.strftime('%H:%M:%S')
        cur.execute('INSERT INTO Attendence_Record (Date, Name, Entry_time) VALUES (?, ?, ?)', (Date, Name, time))
        cur.commit()


    elif key == ord('q'):
        break

df = pd.DataFrame(results)
df.to_csv('Attendence.csv',index=False)

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


In [49]:
cur.execute("select*from attendence_Record")

In [50]:
data = cur.fetchall()
data = np.array(data)
data.shape

(2, 3)

In [51]:
data

array([[datetime.date(2024, 9, 9), 'donald', datetime.time(7, 46, 36)],
       [datetime.date(2024, 9, 9), 'donald', datetime.time(7, 46, 46)]],
      dtype=object)

In [62]:
cur.execute("truncate table Attendence_Record")
cur.commit()
cur.execute("select*from Attendence_Record")
data = cur.fetchall()
data = np.array(data)
data.shape

(0,)